# Data Cleaning and Preprocessing

## Loading the Libraries

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [38]:
data_df = pd.read_csv('/home/amy/work/RIT/TDess/DSCI-601-Amy/Data/Combined/combined_AAPL.csv')

In [39]:
data_df

,date,RET,VOL_CHANGE,BA_SPREAD,ILLIQUIDITY,sprtrn,TURNOVER,DJI_Return
0,1/2/1992,0.055432,0.717745,0.008403,4.510000e-10,0.000408,17.419850,0.000000
1,1/3/1992,-0.008403,-0.172890,0.004237,-8.340000e-11,0.004985,14.408127,0.009173
2,1/6/1992,-0.016949,-0.399632,0.004310,-2.850000e-10,-0.003291,8.650181,-0.000437
3,1/7/1992,0.019397,0.237283,0.004228,2.590000e-10,-0.001340,10.702726,0.001469
4,1/8/1992,0.023256,0.645321,0.004132,1.840000e-10,0.001677,17.609419,-0.000281
...,...,...,...,...,...,...,...,...
7804,12/23/2022,-0.002798,-0.181476,0.000076,-3.330000e-13,0.005868,4.008909,0.005342
7805,12/27/2022,-0.013878,0.081093,0.000231,-1.550000e-12,-0.004050,4.334004,0.001133
7806,12/28/2022,-0.030685,0.238299,0.000079,-2.850000e-12,-0.012021,5.366792,-0.011006
7807,12/29/2022,0.028324,-0.115337,0.000231,2.890000e-12,0.017461,4.747802,0.010497


In [40]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7809 entries, 0 to 7808
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7809 non-null   object 
 1   RET          7809 non-null   float64
 2   VOL_CHANGE   7809 non-null   float64
 3   BA_SPREAD    7809 non-null   float64
 4   ILLIQUIDITY  7809 non-null   float64
 5   sprtrn       7809 non-null   float64
 6   TURNOVER     7809 non-null   float64
 7   DJI_Return   7809 non-null   float64
dtypes: float64(7), object(1)
memory usage: 488.2+ KB


### Changing the date column to timestamp

In [41]:
data_df['date'] = pd.to_datetime(data_df['date'])

In [42]:
print(data_df.isnull().sum())


date           0
RET            0
VOL_CHANGE     0
BA_SPREAD      0
ILLIQUIDITY    0
sprtrn         0
TURNOVER       0
DJI_Return     0
dtype: int64


### Feature Engineering

#### Adding new features might help

In [43]:
data_df['year'] = data_df['date'].dt.year
data_df['month'] = data_df['date'].dt.month
data_df['day_of_week'] = data_df['date'].dt.dayofweek

#### Creating lagged features

Since we need to predict the returns of the stock for a day, we cannot use the feature values of the same day since they will be unavailable at actual inference time. We need to use statistics like mean, standard deviation of their lagged values.

We will use three sets of lagged values, one previous day, one looking back 7 days and another looking back 30 days as a proxy for last week and last month metrics.

*understand how rolling window works.*

In [44]:
# Create lagged features for RET

data_df['RET_lag1'] = data_df['RET'].shift(1)  # Lag of 1 day
data_df['RET_lag7'] = data_df['RET'].shift(7)  # Lag of 7 days
data_df['RET_lag30'] = data_df['RET'].shift(30)  # Lag of 30 days

In [45]:
# Setting window sizes
windows = [1, 7, 30]

# Calculating rolling mean and standard deviation
for window in windows:
    data_df[f'mean_{window}d'] = data_df[f'RET_lag{window}'].rolling(window=window, min_periods=1).mean()
    data_df[f'std_{window}d'] = data_df[f'RET_lag{window}'].rolling(window=window, min_periods=1).std()


In [46]:
data_df

,date,RET,VOL_CHANGE,BA_SPREAD,ILLIQUIDITY,sprtrn,TURNOVER,DJI_Return,year,month,day_of_week,RET_lag1,RET_lag7,RET_lag30,mean_1d,std_1d,mean_7d,std_7d,mean_30d,std_30d
0,1992-01-02,0.055432,0.717745,0.008403,4.510000e-10,0.000408,17.419850,0.000000,1992,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-01-03,-0.008403,-0.172890,0.004237,-8.340000e-11,0.004985,14.408127,0.009173,1992,1,4,0.055432,NaN,NaN,0.055432,NaN,NaN,NaN,NaN,NaN
2,1992-01-06,-0.016949,-0.399632,0.004310,-2.850000e-10,-0.003291,8.650181,-0.000437,1992,1,0,-0.008403,NaN,NaN,-0.008403,NaN,NaN,NaN,NaN,NaN
3,1992-01-07,0.019397,0.237283,0.004228,2.590000e-10,-0.001340,10.702726,0.001469,1992,1,1,-0.016949,NaN,NaN,-0.016949,NaN,NaN,NaN,NaN,NaN
4,1992-01-08,0.023256,0.645321,0.004132,1.840000e-10,0.001677,17.609419,-0.000281,1992,1,2,0.019397,NaN,NaN,0.019397,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7804,2022-12-23,-0.002798,-0.181476,0.000076,-3.330000e-13,0.005868,4.008909,0.005342,2022,12,4,-0.023773,-0.015536,0.088975,-0.023773,NaN,-0.003260,0.015675,0.001529,0.031193
7805,2022-12-27,-0.013878,0.081093,0.000231,-1.550000e-12,-0.004050,4.334004,0.001133,2022,12,1,-0.002798,-0.046854,0.019269,-0.002798,NaN,-0.006329,0.021679,0.003172,0.030768
7806,2022-12-28,-0.030685,0.238299,0.000079,-2.850000e-12,-0.012021,5.366792,-0.011006,2022,12,2,-0.013878,-0.014579,-0.009486,-0.013878,NaN,-0.006443,0.021727,0.001831,0.030399
7807,2022-12-29,0.028324,-0.115337,0.000231,2.890000e-12,0.017461,4.747802,0.010497,2022,12,3,-0.030685,-0.015910,0.011869,-0.030685,NaN,-0.010449,0.020267,0.001372,0.030131


### Feature Scaling

In [24]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
data_df[['RET', 'VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'sprtrn', 'TURNOVER', 'DJI_Return']] = scaler.fit_transform(data_df[['RET', 'VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'sprtrn', 'TURNOVER', 'DJI_Return']])

In [25]:
data_df

,date,RET,VOL_CHANGE,BA_SPREAD,ILLIQUIDITY,sprtrn,TURNOVER,DJI_Return,year,month,day_of_week
0,1992-01-02,0.674670,0.093796,0.576178,0.541621,0.510306,0.038988,0.532141,1992,1,3
1,1992-01-03,0.599656,0.040361,0.441415,0.443204,0.529730,0.031372,0.569902,1992,1,4
2,1992-01-06,0.589613,0.026758,0.443778,0.406077,0.494608,0.016810,0.530341,1992,1,0
3,1992-01-07,0.632324,0.064970,0.441125,0.506262,0.502888,0.022001,0.538187,1992,1,1
4,1992-01-08,0.636859,0.089451,0.438017,0.492449,0.515691,0.039468,0.530985,1992,1,2
...,...,...,...,...,...,...,...,...,...,...,...
7804,2022-12-23,0.606242,0.039846,0.306800,0.458502,0.533477,0.005072,0.554133,2022,12,4
7805,2022-12-27,0.593222,0.055599,0.311816,0.458278,0.491387,0.005894,0.536806,2022,12,1
7806,2022-12-28,0.573471,0.065031,0.306913,0.458039,0.457560,0.008506,0.486834,2022,12,2
7807,2022-12-29,0.642815,0.043814,0.311835,0.459096,0.582674,0.006941,0.575353,2022,12,3
